# 深圳二手房成价预测

In [1]:
print("ok")

ok


In [ ]:
import requests, time, re, csv, os, pandas_profiling
from bs4 import BeautifulSoup as BS
from multiprocessing import Pool
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from urllib.request import quote
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, \
    precision_score, recall_score, roc_auc_score

### 3.1 特征选取

In [2]:

df3 = pd.read_csv('data.csv').reset_index(drop=True)
df3[['经度', '纬度']] = df3[['经度', '纬度']].round(6)

In [187]:
def get_poi(row, search, radius):
    search = quote(search)
    lat_lng = str(row['纬度']) + ',' + str(row['经度'])
    radius = radius
    ak = 213
    url = f'http://api.map.baidu.com/place/v2/search?query={search}&location={lat_lng}&radius={radius}&output=json&ak={ak}'
    result = requests.get(url)
    return len(result.json()['results'])


for s in ['地铁站', '公交车站', '三甲医院', '小学']:
    for r in [500, 1000, 2000]:
        poi[f'{s}_{r}m'] = poi.apply(get_poi, search=s, radius=r, axis=1)


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs


**(2) 原数据集字段选取**

In [188]:
df3 = df3.drop(['成交时间', '调价', '带看', '关注', '浏览', '房屋年限', '挂牌时间', '梯户比例', '交易权属',
                '建筑结构', '挂牌价格', '成交单价', '成交周期', '区域', '子区域', '套内面积', '楼盘名称'], axis=1)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8802 entries, 0 to 8801
Data columns (total 16 columns):
经度      8802 non-null float64
纬度      8802 non-null float64
成交价     8802 non-null int64
房屋户型    8802 non-null object
所在楼层    8802 non-null object
建筑面积    8802 non-null float64
户型结构    7810 non-null object
建筑类型    8737 non-null object
房屋朝向    8802 non-null object
建成年代    8786 non-null float64
装修情况    8802 non-null object
产权年限    8802 non-null object
配备电梯    8556 non-null object
房屋用途    8802 non-null object
房权所属    7897 non-null object
楼层数     8802 non-null int64
dtypes: float64(4), int64(2), object(10)
memory usage: 1.1+ MB


**(3) 合成建模数据集**

In [3]:
poi = pd.read_csv('poi.csv').drop('楼盘名称', axis=1)
poi[['经度', '纬度']] = poi[['经度', '纬度']].round(6)
df_merge = pd.merge(df3, poi, how='left', on=['经度', '纬度'])
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8802 entries, 0 to 8801
Data columns (total 45 columns):
楼盘名称          8802 non-null object
经度            8802 non-null float64
纬度            8802 non-null float64
成交时间          8802 non-null object
成交价           8802 non-null int64
成交单价          8802 non-null float64
区域            8802 non-null object
子区域           8800 non-null object
挂牌价格          8802 non-null float64
成交周期          8802 non-null float64
调价            8802 non-null int64
带看            8802 non-null int64
关注            8802 non-null int64
浏览            8640 non-null float64
房屋户型          8802 non-null object
所在楼层          8802 non-null object
建筑面积          8802 non-null float64
户型结构          7810 non-null object
套内面积          4688 non-null float64
建筑类型          8737 non-null object
房屋朝向          8802 non-null object
建成年代          8786 non-null float64
装修情况          8802 non-null object
建筑结构          8802 non-null object
梯户比例          8802 non-null object
产权年限         

### 3.2 数据预处理&查看

**(1) 数据预处理函数构建**

In [190]:
# 异常值处理
def show_error(df, col, whis=1.5, show=False):
    iqr = df[col].quantile(0.75) - df[col].quantile(0.25)
    upper_bound = df[col].quantile(0.75) + whis * iqr  # 上界
    lower_bound = df[col].quantile(0.25) - whis * iqr  # 下界
    print('【', col, '】上界异常值总数：', df[col][df[col] > upper_bound].count())
    if show:
        print('异常值示例：\n', df[df[col] > upper_bound].head(5).T)
    print('【', col, '】下界异常值总数：', df[col][df[col] < lower_bound].count())
    if show:
        print('异常值示例：\n', df[df[col] < lower_bound].head(5).T)
    print('- - - - - - ')


def drop_error(df, col, whis=1.5):
    iqr = df[col].quantile(0.75) - df[col].quantile(0.25)
    upper_bound = df[col].quantile(0.75) + whis * iqr  # 上界
    lower_bound = df[col].quantile(0.25) - whis * iqr  # 下界
    data_del = df[col][(df[col] > upper_bound) | (df[col] < lower_bound)].count()
    data = df[(df[col] <= upper_bound) & (df[col] >= lower_bound)]
    print(f'【{col}】总剔除数据量：', data_del)
    return data.reset_index(drop=True)

In [1]:
# 空值处理
def fill_with_neighbor_or_mode(df, col):
    loc = df[df[col].isna()][['经度', '纬度']]
    loclist = set((l[1], l[2]) for l in loc.itertuples())
    mode_number = df[col].mode()[0]
    for i in loclist:
        try:
            r = df[(df.经度 == i[0]) & (df.纬度 == i[1])][col].mode()[0]
            df.loc[(df.经度 == i[0]) & (df.纬度 == i[1]) & df[col].isna(), col] = r
        except Exception as e:
            df.loc[(df.经度 == i[0]) & (df.纬度 == i[1]) & df[col].isna(), col] = mode_number
    print(f'fill_with_neighbor_or_mode - {col} , done!')

In [2]:
# 房屋户型哑变量获取
def roomtype_get_dummies(df):
    roomtype = df.房屋户型.str.extract('(?P<室>\d+)室(?P<厅>\d+)厅(?P<厨>\d+)厨(?P<卫>\d+)卫').astype(np.int8)
    print('roomtype_get_dummies , done!')
    return pd.merge(df, roomtype, how='left', left_index=True, right_index=True).drop('房屋户型', axis=1)

In [3]:
# 房屋朝向哑变量获取
def orientation_get_dummies(df):
    l = set()
    for i in df.房屋朝向.str.split(' '):
        l.update(i)
    d = pd.DataFrame(np.zeros((len(df), len(l)), dtype=np.int8),
                     columns=[f'房屋朝向_{i}' for i in l])
    df = df.join(d)
    for n in l:
        df.loc[df.房屋朝向.str.contains(n), f'房屋朝向_{n}'] = 1
    print('已获取房屋朝向哑变量')
    return df.drop('房屋朝向', axis=1)

In [4]:
# 其他哑变量获取
def dummies_getting(df, col):
    df = pd.get_dummies(df, columns=col)
    print(f'dummies_getting - {col} , done!')
    return df

**(2) 数据预处理&查看**

In [195]:
# 先处理空值
df_merge.loc[df_merge.产权年限 == '未知', '产权年限'] = np.nan
for i in ['户型结构', '建筑类型', '建成年代', '配备电梯', '房权所属', '产权年限']:
    fill_with_neighbor_or_mode(df_merge, i)
df_merge.产权年限 = df_merge.产权年限.str[:2].astype(np.int8)

fill_with_neighbor_or_mode - 户型结构 , done!
fill_with_neighbor_or_mode - 建筑类型 , done!
fill_with_neighbor_or_mode - 建成年代 , done!
fill_with_neighbor_or_mode - 配备电梯 , done!
fill_with_neighbor_or_mode - 房权所属 , done!
fill_with_neighbor_or_mode - 产权年限 , done!


In [196]:
# 查看一下最终会保留的连续值字段异常值情况，注意这里whis是3
plt.figure(figsize=(16, 9))
for n, i in enumerate(['成交价', '建筑面积', '楼层数', '建成年代']):
    plt.subplot(1, 4, n + 1)
    plt.title(i)
    sns.boxplot(df_merge[i], orient='v', width=0.2, whis=3)
    plt.ylabel('')
for i in ['成交价', '建筑面积', '楼层数', '建成年代']:
    show_error(df_merge, i, whis=3)

【 成交价 】上界异常值总数： 192
【 成交价 】下界异常值总数： 0
- - - - - - 
【 建筑面积 】上界异常值总数： 31
【 建筑面积 】下界异常值总数： 0
- - - - - - 
【 楼层数 】上界异常值总数： 0
【 楼层数 】下界异常值总数： 0
- - - - - - 
【 建成年代 】上界异常值总数： 0
【 建成年代 】下界异常值总数： 0
- - - - - - 


<Figure size 1152x648 with 4 Axes>

In [197]:
# 处理异常值 
for i in ['成交价', '建筑面积', '楼层数', '建成年代']:
    df_merge = drop_error(df_merge, i, whis=3)  # 注意这里whis是3

# 房屋户型哑变量获取
df_merge = roomtype_get_dummies(df_merge)

# 房屋朝向哑变量获取
df_merge = orientation_get_dummies(df_merge)

# 其他哑变量获取
columns = ['所在楼层', '户型结构', '建筑类型', '装修情况', '配备电梯', '房屋用途', '房权所属']
df_merge = dummies_getting(df_merge, columns)

【成交价】总剔除数据量： 192
【建筑面积】总剔除数据量： 3
【楼层数】总剔除数据量： 0
【建成年代】总剔除数据量： 0
roomtype_get_dummies , done!
orientation_get_dummies , done!
dummies_getting - ['所在楼层', '户型结构', '建筑类型', '装修情况', '配备电梯', '房屋用途', '房权所属'] , done!


In [198]:
plt.figure(figsize=(16, 9))
for n, i in enumerate(['成交价', '建筑面积', '楼层数', '建成年代']):
    plt.subplot(4, 1, n + 1)
    plt.title(i)
    sns.distplot(df_merge[i])
    plt.ylabel('')

<Figure size 1152x648 with 4 Axes>

整体分布偏态情况减小

### 3.3 训练模型

**(1) 数据准备**

In [218]:
# 准备训练、测试集
X = df_merge.drop(['成交价'], axis=1)
y = df_merge['成交价']
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)  # random_state=42

# k折交叉拆分器 - 用于网格搜索
cv = KFold(n_splits=3, shuffle=True)

In [219]:
# 回归模型性能查看函数
def perfomance_reg(model, X, y, name=None):
    y_predict = model.predict(X)
    check = pd.DataFrame(y)
    check['y_predict'] = y_predict
    check['abs_err'] = abs(check['y_predict'] - check[y.name])
    check['ape'] = check['abs_err'] / check[y.name]
    ape = check['ape'][check['ape'] != np.inf].mean()
    if name:
        print(name, ':')
    print(f'mean squared error is: {mean_squared_error(y, y_predict)}')
    print(f'mean absolute error is: {mean_absolute_error(y, y_predict)}')
    print(f'R Squared is: {r2_score(y, y_predict)}')
    print(f'mean absolute percent error is: {ape}')
    print('- - - - - - ')

**(2) lasso回归模型**

In [220]:
# 定义LassoCV（可自动通过Cross Validation搜索最佳正则系数alpha）
pipe_lassocv = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('sc', StandardScaler()),
    ('pwt', PowerTransformer()),
    ('lasso_regr', LassoCV(
        # 待搜索alpha值如下：
        # alphas=(list(np.arange(1,10)*0.1)+list(np.arange(1,10))+list(np.arange(1,11)*10))
        # alpha搜索完后发现0.1最合适
        alphas=(list(np.arange(1, 10) * 0.1) + list(np.arange(1, 10)) + list(np.arange(1, 11) * 10)),
        cv=KFold(n_splits=3, shuffle=True),  # 配合搜索参数使用
        n_jobs=-1, verbose=1))
])

# 搜索参数并训练模型
pipe_lassocv.fit(xtrain, ytrain)
# 最佳参数组合
print('最佳alpha值：', pipe_lassocv.named_steps['lasso_regr'].alpha_)
# 训练集性能指标
perfomance_reg(pipe_lassocv, xtrain, ytrain, name='train')
# 测试集性能指标
perfomance_reg(pipe_lassocv, xtest, ytest, name='test')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
....................................................................................[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   14.4s finished


最佳alpha值： 0.1
train :
mean squared error is: 5866.1306605690725
mean absolute error is: 52.63572482286287
R Squared is: 0.8142003463697659
mean absolute percent error is: 0.15272275854277395
- - - - - - 
test :
mean squared error is: 6940.790079075325
mean absolute error is: 56.64143852727948
R Squared is: 0.7837884925316358
mean absolute percent error is: 0.16194771064655833
- - - - - - 


**(3) 随机森林回归模型**

In [221]:
# 定义RandomForestRegressor随机森林回归模型
rf_reg = RandomForestRegressor(criterion='mse',
                               n_jobs=-1, )  # random_state
# 参数设定
rf_grid_params = {'max_features': ['auto'],  # ['auto',0.5,0.6,0.9] 带搜索参数
                  'max_depth': [8, 9, 10],  # [3,6,9]
                  'n_estimators': [500, 800, 1000]}
# 参数搜索
rf_gridsearch = GridSearchCV(rf_reg, rf_grid_params, cv=cv,
                             n_jobs=-1, scoring='neg_mean_squared_error', verbose=5, refit=True)
# 工作流管道
pipe_rf = Pipeline([
    ('sc', StandardScaler()),
    ('rf_grid', rf_gridsearch)
])

# 搜索参数并训练模型
pipe_rf.fit(xtrain, ytrain)
# 最佳参数组合
print('最佳alpha值：', pipe_rf.named_steps['rf_grid'].best_params_)
# 训练集性能指标
perfomance_reg(pipe_rf, xtrain, ytrain, name='train')
# 测试集性能指标
perfomance_reg(pipe_rf, xtest, ytest, name='test')

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  7.1min finished


最佳alpha值： {'max_depth': 10, 'max_features': 'auto', 'n_estimators': 500}
train :
mean squared error is: 1418.8145215140926
mean absolute error is: 24.641916388425237
R Squared is: 0.9550614771616268
mean absolute percent error is: 0.06822412461801346
- - - - - - 
test :
mean squared error is: 3337.751860311248
mean absolute error is: 35.17980895328542
R Squared is: 0.896026193984911
mean absolute percent error is: 0.09469746708246876
- - - - - - 


**(4) xgboost模型**

In [224]:
# xgboost模型
xgb_reg = xgb.XGBRegressor(objective='reg:linear',
                           n_job=-1,
                           booster='gbtree',
                           learning_rate=0.05)
# 参数设定
xgb_params = {'max_depth': [6, 9],
              'subsample': [0.6, 0.9, 1],
              'colsample_bytree': [0.5, 0.6],
              'reg_alpha': [0, 0.05, 0.1],
              'n_estimators': [750, 1000]}
# 参数搜索
xgb_gridsearch = GridSearchCV(xgb_reg, xgb_params, cv=cv, n_jobs=-1,
                              verbose=10, refit=True)
# 工作流管道
pipe_xgb = Pipeline([
    ('sc', StandardScaler()),
    ('xgb_grid', xgb_gridsearch)
])

# 搜索参数并训练模型
pipe_xgb.fit(xtrain, ytrain)
# 最佳参数组合
print(pipe_xgb.named_steps['xgb_grid'].best_params_)
# 训练集性能指标
perfomance_reg(pipe_xgb, xtrain, ytrain, name='train')
# 测试集性能指标
perfomance_reg(pipe_xgb, xtest, ytest, name='test')

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed: 15

{'colsample_bytree': 0.6, 'max_depth': 6, 'n_estimators': 1000, 'reg_alpha': 0, 'subsample': 0.6}
train :
mean squared error is: 159.2255553231461
mean absolute error is: 9.285820319259521
R Squared is: 0.9949568029183222
mean absolute percent error is: 0.028887646995371632
- - - - - - 
test :
mean squared error is: 1962.5934603322894
mean absolute error is: 24.419883692583003
R Squared is: 0.9388635464015461
mean absolute percent error is: 0.06388880018075778
- - - - - - 


**(5) 集成调优**

In [229]:
# 获取xgboost模型各特征重要程度数值
importances = pipe_xgb.named_steps['xgb_grid'].best_estimator_.feature_importances_
features = xtrain.columns
importance_table = pd.DataFrame({'features': features, 'importances': importances})
importance_table.sort_values(by='importances', ascending=False)

,features,importances
2,建筑面积,0.160504
0,经度,0.154278
1,纬度,0.121446
3,建成年代,0.068596
5,楼层数,0.064920
16,小学_1000m,0.034107
8,地铁站_2000m,0.031472
9,公交车站_500m,0.030303
18,室,0.027264
7,地铁站_1000m,0.026159


In [230]:
# 以重要程度中位数以上选取特征名称
im_median = importance_table.importances.median()
select_list = importance_table[importance_table.importances >= im_median].features.tolist()
select_list.extend(['ensemble_rf', '成交价'])  # '成交价'为方便后续拆分数据集；'ensemble_rf'为随机森林预测结果
select_list

['经度',
 '纬度',
 '建筑面积',
 '建成年代',
 '楼层数',
 '地铁站_500m',
 '地铁站_1000m',
 '地铁站_2000m',
 '公交车站_500m',
 '三甲医院_2000m',
 '小学_500m',
 '小学_1000m',
 '小学_2000m',
 '室',
 '厅',
 '卫',
 '房屋朝向_南',
 '房屋朝向_东',
 '房屋朝向_北',
 '所在楼层_中楼层',
 '所在楼层_低楼层',
 '所在楼层_高楼层',
 '建筑类型_塔楼',
 '装修情况_其他',
 '装修情况_简装',
 '装修情况_精装',
 '房权所属_共有',
 'ensemble_rf',
 '成交价']

In [231]:
df_ensemble = df_merge.copy()
df_ensemble['ensemble_rf'] = pipe_rf.predict(df_ensemble.drop('成交价', axis=1))
df_ensemble = df_ensemble.loc[:, select_list]

In [232]:
# 生成集成学习训练、验证集合
XX = df_ensemble.drop('成交价', axis=1)
yy = df_ensemble['成交价']
xxtrain, xxtest, yytrain, yytest = train_test_split(XX, yy, test_size=0.2, random_state=1)  # random_state=42

pipe_xgb_ensemble = Pipeline([
    ('sc', StandardScaler()),
    ('xgb_grid', xgb_gridsearch)
])

# 搜索参数并训练模型
pipe_xgb_ensemble.fit(xxtrain, yytrain)
# 最佳参数组合
print(pipe_xgb_ensemble.named_steps['xgb_grid'].best_params_)
# 训练集性能指标
perfomance_reg(pipe_xgb_ensemble, xxtrain, yytrain, name='train')
# 测试集性能指标
perfomance_reg(pipe_xgb_ensemble, xxtest, yytest, name='test')

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed: 10

{'colsample_bytree': 0.6, 'max_depth': 6, 'n_estimators': 1000, 'reg_alpha': 0, 'subsample': 0.9}
train :
mean squared error is: 89.51619788727905
mean absolute error is: 7.000624772614124
R Squared is: 0.9971662290176454
mean absolute percent error is: 0.022116629050790163
- - - - - - 
test :
mean squared error is: 1105.3433054824618
mean absolute error is: 19.5376985707211
R Squared is: 0.9654939961043317
mean absolute percent error is: 0.052662943951393294
- - - - - - 


In [ ]:
# 利用joblib存储模型
from sklearn.externals import joblib

joblib.dump({'model_name': 'pipe_xgb_ensemble',
             'features_list': select_list,
             'model': pipe_xgb_ensemble,
             'data': [xxtrain, xxtest, yytrain, yytest]},
            'XGB.pkl')